# Introdução.

* A Rossmann opera mais de 3.000 drogarias em 7 países europeus. Atualmente, os gerentes de loja da Rossmann têm a tarefa de prever suas vendas diárias com até seis semanas de antecedência. As vendas da loja são influenciadas por muitos fatores, incluindo promoções, competição, feriados escolares e estaduais, sazonalidade e localidade. Com milhares de gerentes individuais prevendo vendas com base em suas circunstâncias únicas, a precisão dos resultados pode ser bastante variada.

# Descrição do projeto
 
  * Após a exclusão e modificação de algumas variáveis, foi visto que o desempenho do modelo padrão de machine learning passou de um MAPE de 5.86092 Std: 0.96943 Erro: 0.43354  para um de 5.79653 Std: 0.93450 Erro: 0.41792. E o número de variáveis foi de 32 para 20.

* Logo, nessa parte deste trabalho será visto o turing dos parâmetros de diversos modelos de machine learning.
 
* A estratégia utilizada para realizar o que foi proposto, segue os seguintes passos:
 * Separação dos dados em Teste e Validação. 
   * Validação: 2013/01/01 - 2015/06/18.
   * Teste: 2015/06/19 - 2015/07/31.
   * A Validação será feita para tunar os modelos de machine learning e ela será feita método prequential Expanding.
* Os modelos serão avaliados nas seguintes métricas:
   * MAPE - Erro médio Absoluto Percentual.
   * RMSLE - Raiz do erro médio quadrático considerando o log.
   * RMSE - Raiz do erro médio quadrático.
 * Modelos utilizados:
   * Random Forest.
   * LGBM.
   * XGBM
 
 
--- 
   * **Os dados de teste serão utilizados para simular o ambiente real no qual o modelo de machine learning será aplicado. (Parte final para decidir qual o melhor modelo)**
--- 
 
 

# Descrição dos dados.

* Os dados foram obtidos do kaggle: 
  * https://www.kaggle.com/c/rossmann-store-sales


* Descrição dos arquivos utilizados :
 
 * **train.csv** - historical data including Sales
 * **test.csv** - historical data excluding Sales
 * **store.csv** - supplemental information about the stores
 
* **Os dados utilizados veio do tratamento feito nos arquivos train,test,store**. 

* Legenda:
  * **[T]** - Dados veio do arquivo Train.
  * **[S]** - Dados veio do arquivo do Store.
  * **[C]** - Criado pelo notebook Tratamento dos dados.
  * **[D]** - Criado pelo notebook Descrição dos dados.
  * **[M]** - Criado pelo notebook Modification dataset
  * **[CD]** - Criado pelo notebook Codification dataset  

* Descrição dos dados.
 
 * **day_of_week [T]** - Day of week. 
 * **date [T]** - days the data was acquired. 
 * **store [T]** - a unique Id for each store 
 * **sales [T]** - the turnover for any given day (this is what you are predicting)  
 * **customers [T]** - the number of customers on a given day
 * **store_type_a [M]** - store models a. 
 * **store_type_b [M]** - store models b. 
 * **store_type_c [M]** - store models c.  
 * **competition_distance [S]** - distance in meters to the nearest competitor store.
 * **competition_open_since_year [S]** - gives the approximate year of the time the nearest competitor was opened.
 * **Promo [T]** - indicates whether a store is running a promo on that day
 * **Promo2 [S]** - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating.
 * **promo2_since_year [S]** - describes the year  when the store started participating in Promo2.
 * **year [C]** - year the data died. 
 * **week_year [C]**  - calendar week  the data died.
 * **without_competition [C]** - Store has competition. 
 * **promo_time_week [C]** - Time the store works without the promotion.
 * **Total_dias [CD]** - Vccumulated days.
 * **day_cos [CD]** - Day variable transformed into cosine sign.
 * **holiday [CD]** - The day is a holiday or not. Where 1 is a holiday and 0 is a day that is not.
 * **assortment [CD]** - Describe assortment level the store. 1 - > Extended, 2 ->  Estra, 3 -> basic.


# Importando bibliotecas.

In [1]:
!pip install git+git://github.com/Hvass-Labs/scikit-optimize.git@dd7433da068b5a2509ef4ea4e5195458393e6555`

/usr/bin/sh: 1: Syntax error: EOF in backquote substitution


In [2]:
!pip install lightgbm

# Funções

In [3]:
def MAPE(y_pred,Y_teste):
  return np.mean(100*np.abs(y_pred - Y_teste)/np.abs(Y_teste))

def label_encoder(d,col):
  enc      = LabelEncoder()
  enc      = enc.fit(d[col])
  inteiros = enc.fit_transform(d[col])
  d[col]   = inteiros
  return d

def dummies_var(dados,coluna):
  df_aux = dados
  df_aux = df_aux.join(pd.get_dummies(df_aux[coluna], prefix=coluna))
  df_aux = df_aux.drop(columns=coluna)
  return df_aux




def validation(X,Y,coluna,flag,Date,model):
  vetor_MAPE = np.zeros(5) 
  vetor_MSLR = np.zeros(5)
  vetor_RMSE = np.zeros(5)
  Y_pred = []
  Y_val  = []
  for i in range(5):
    # Dados de treinamento
    
    X_t = X[X['Total_dias']  < 731 + 6*7*i]
    y_t = Y[coluna][Y['Total_dias']  < 731 + 6*7*i]
    X_t = X_t.drop(columns = ['Total_dias'])
    
    X_v = X[(X['Total_dias'] >= 731 + 6*7*i) & (X['Total_dias'] <= 731 + 6*7*(i+1))]
    y_v = Y[coluna][(Y['Total_dias'] >= 731 + 6*7*i) & (Y['Total_dias'] <= 731 + 6*7*(i+1))]
    X_v = X_v.drop(columns = ['Total_dias'])
    
    md        = model.fit(X_t, y_t)
    y_pred    = md.predict(X_v)
    
    if flag == 'log':
      y_pred = np.round(10**y_pred - 1)  # eliminar o ruído provocado pelo computados atravez dou round
      y_v    = np.round(10**y_v - 1)
    if flag == 'sqrt':
      y_pred = np.round(y_pred**2)
      y_v    = np.round(y_v**2)
    if flag == 'normal':
      y_pred = y_pred
      y_v    = y_v
    
  
    
    mape = MAPE(y_v,y_pred)
    mslr = mean_squared_log_error(y_v,y_pred)
    rmse = mean_squared_error(y_v,y_pred,squared =False)
    vetor_MAPE[i] = mape
    vetor_MSLR[i] = mslr
    vetor_RMSE[i] = rmse
  
    
  return np.mean(vetor_MAPE)


def validation1(X,Y,coluna,flag,Date,model):
  vetor_MAPE = np.zeros(5) 
  vetor_MSLR = np.zeros(5)
  vetor_RMSE = np.zeros(5)
  Y_pred = []
  Y_val  = []
  for i in range(5):
    # Dados de treinamento
    print('Data do inicio do treinamento:', Date[X['Total_dias'] < 731 + 6*7*i].dt.date.values[0],
          'Data do final do treinamento :',Date[X['Total_dias'] < 731 + 6*7*i].dt.date.values[-1])
  
    
    X_t = X[X['Total_dias']  < 731 + 6*7*i]
    y_t = Y[coluna][Y['Total_dias']  < 731 + 6*7*i]
    X_t = X_t.drop(columns = ['Total_dias'])
    
    # Dados de validação
    print('Data do inicio do Validação  :', Date[(X['Total_dias'] >= 731 + 6*7*i) & (X['Total_dias'] <= 731 + 6*7*(i+1))].dt.date.values[0],
          'Data do final do Validação   :',Date[(X['Total_dias'] >= 731 + 6*7*i) & (X['Total_dias'] <= 731 + 6*7*(i+1))].dt.date.values[-1])

    X_v = X[(X['Total_dias'] >= 731 + 6*7*i) & (X['Total_dias'] <= 731 + 6*7*(i+1))]
    y_v = Y[coluna][(Y['Total_dias'] >= 731 + 6*7*i) & (Y['Total_dias'] <= 731 + 6*7*(i+1))]
    X_v = X_v.drop(columns = ['Total_dias'])
    
    md        = model.fit(X_t, y_t)
    y_pred    = md.predict(X_v)
    
    if flag == 'log':
      y_pred = np.round(10**y_pred - 1)  # eliminar o ruído provocado pelo computados atravez dou round
      y_v    = np.round(10**y_v - 1)
    if flag == 'sqrt':
      y_pred = np.round(y_pred**2)
      y_v    = np.round(y_v**2)
    if flag == 'normal':
      y_pred = y_pred
      y_v    = y_v
    
  
    
    mape = MAPE(y_v,y_pred)
    mslr = mean_squared_log_error(y_v,y_pred)
    rmse = mean_squared_error(y_v,y_pred,squared =False)
    vetor_MAPE[i] = mape
    vetor_MSLR[i] = mslr
    vetor_RMSE[i] = rmse
  
    print('MAPE: ',np.format_float_positional(mape,5))
    print('RMSLE:',np.format_float_positional(mslr,5))
    print('RMSE: ',np.format_float_positional(rmse,5))
     
   
  print('Mape Geral: ',np.format_float_positional(np.mean(vetor_MAPE),5),'Std:',np.format_float_positional(np.std(vetor_MAPE),5),'Erro:',np.format_float_positional(np.std(vetor_MAPE)/np.sqrt(5),5))
  print('Rmsle Geral:',np.format_float_positional(np.mean(vetor_MSLR),5),'Std:',np.format_float_positional(np.std(vetor_MSLR),5),'Erro:',np.format_float_positional(np.std(vetor_MSLR)/np.sqrt(5),5))
  print('Rmse Geral: ',np.format_float_positional(np.mean(vetor_RMSE),5),'Std:',np.format_float_positional(np.std(vetor_RMSE),5),'Erro:',np.format_float_positional(np.std(vetor_RMSE)/np.sqrt(5),5))
  return vetor_MAPE,vetor_MSLR,vetor_RMSE

def Tuning_RF(params):
  max_depth = params[0]
  n_estimators = params[1]  
  model = RandomForestRegressor(n_estimators = n_estimators,max_depth=max_depth,random_state = 10) # modelo padrão
  mape = validation(X,Y,'sales','normal',Date,model)
  return mape


def Tuning_LGBM(params):
  lr = params[0]
  max_depth = params[1]
  n_estimators = params[2]

  print('lr:',params[0])
  print('max_depth:',params[1])
  print('n_estimators:',params[2])
  
  model = LGBMRegressor(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         n_estimators=n_estimators, random_state=40, 
                         n_jobs=6)

  mape = validation(X,Y,'sales','normal',Date,model)
  return mape

# Biblioteca

In [4]:
import pandas as pd
#from google.colab import drive
#drive.mount('/content/drive')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error,mean_squared_error
import skopt
from skopt import gp_minimize, forest_minimize
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder

# Importando os dados.

In [21]:
df = pd.read_csv('dados_novos3.csv')

In [22]:
df.head() # vendo os dados

,Unnamed: 0,store,day_of_week,date,sales,customers,promo,competition_distance,competition_open_since_year,promo2,...,year,week_year,promo_time_week,Total_dias,store_type_a,store_type_b,store_type_c,day_cos,holiday,assortment
0,0,1115,4,2013-01-03,4297,300,0,5350.0,2013,1,...,2013,1,32,3.0,0,0,0,0.809017,0.0,1.0
1,1,1114,4,2013-01-03,18463,3211,0,870.0,2013,0,...,2013,1,0,3.0,1,0,0,0.809017,0.0,1.0
2,2,1113,4,2013-01-03,5563,718,0,9260.0,2013,0,...,2013,1,0,3.0,1,0,0,0.809017,0.0,1.0
3,3,1112,4,2013-01-03,8716,809,0,1880.0,2006,0,...,2013,1,0,3.0,0,0,1,0.809017,0.0,1.0
4,4,1111,4,2013-01-03,4579,444,0,1900.0,2014,1,...,2013,1,-30,3.0,1,0,0,0.809017,0.0,0.0


In [23]:
df = df.drop(columns = ['Unnamed: 0'])

In [24]:
df.head()

,store,day_of_week,date,sales,customers,promo,competition_distance,competition_open_since_year,promo2,promo2_since_year,...,year,week_year,promo_time_week,Total_dias,store_type_a,store_type_b,store_type_c,day_cos,holiday,assortment
0,1115,4,2013-01-03,4297,300,0,5350.0,2013,1,2012,...,2013,1,32,3.0,0,0,0,0.809017,0.0,1.0
1,1114,4,2013-01-03,18463,3211,0,870.0,2013,0,2013,...,2013,1,0,3.0,1,0,0,0.809017,0.0,1.0
2,1113,4,2013-01-03,5563,718,0,9260.0,2013,0,2013,...,2013,1,0,3.0,1,0,0,0.809017,0.0,1.0
3,1112,4,2013-01-03,8716,809,0,1880.0,2006,0,2013,...,2013,1,0,3.0,0,0,1,0.809017,0.0,1.0
4,1111,4,2013-01-03,4579,444,0,1900.0,2014,1,2013,...,2013,1,-30,3.0,1,0,0,0.809017,0.0,0.0


In [25]:
df['date'] = pd.to_datetime(df['date'])

In [26]:
df.index = np.arange(df.shape[0])

# Valor MAPE do modelo padrão

In [27]:
Date = df['date']

In [28]:
df.index = np.arange(df.shape[0])
X        = df.drop(columns = ['date','sales'])
Y        = df[['sales','Total_dias']]

In [29]:
model    = RandomForestRegressor(n_estimators = 10,random_state = 10) 

In [30]:
validation(X,Y,'sales','normal',Date,model)

5.425810961139628

# Tunig dos modelos



### Random Forest.

In [17]:
space = [(1,15), # max_depth
        (10,100)] #n_estimator


In [18]:
res1 = gp_minimize(Tuning_RF, space, random_state=40, n_calls=12, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 262.3326
Function value obtained: 10.6832
Current minimum: 10.6832
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 310.6801
Function value obtained: 9.8713
Current minimum: 9.8713
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 255.4008
Function value obtained: 11.4398
Current minimum: 9.8713
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 406.6018
Function value obtained: 18.3430
Current minimum: 9.8713
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 163.4059
Function value obtained: 20.7827
Current minimum: 9.8713
Iteration No: 6 star

In [19]:
print('Best Value:',res1.fun)
print('Parâmetros:',res1.x)

Best Value: 6.905368256446421
Parâmetros: [15, 100]


* Claramente o resultado não melhora com os parâmetros utilizados, logo será usado o modelo padrão.

In [20]:
model                                = RandomForestRegressor(n_estimators = 10,random_state = 10) 
vetor_MAPE1,vetor_MSLR1,vetor_RMSE1  = validation1(X,Y,'sales','normal',Date,model)

Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2014-12-31
Data do inicio do Validação  : 2015-01-01 Data do final do Validação   : 2015-02-12
MAPE:  5.55816
RMSLE: 0.00559
RMSE:  535.1598
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-02-11
Data do inicio do Validação  : 2015-02-12 Data do final do Validação   : 2015-03-26
MAPE:  4.86899
RMSLE: 0.00405
RMSE:  422.43212
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-03-25
Data do inicio do Validação  : 2015-03-26 Data do final do Validação   : 2015-05-07
MAPE:  5.61305
RMSLE: 0.00535
RMSE:  630.99217
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-05-06
Data do inicio do Validação  : 2015-05-07 Data do final do Validação   : 2015-06-18
MAPE:  5.16878
RMSLE: 0.00450
RMSE:  500.602
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-06-17
Data do inicio do Validação  : 2015-06-18 Data do final do 

### LGBM

In [21]:
space = [(1e-3, 1e-1), # lr
          (1, 10), # max_depth
          (10,200), # n_estimators
          ]

res2 = gp_minimize(Tuning_LGBM, space, random_state=0, n_calls=20, verbose=1)


Iteration No: 1 started. Evaluating function at random point.
lr: 0.05969161720427683
max_depth: 9
n_estimators: 173
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 91.5108
Function value obtained: 5.7689
Current minimum: 5.7689
Iteration No: 2 started. Evaluating function at random point.
lr: 0.08487792213962844
max_depth: 7
n_estimators: 83
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 31.4302
Function value obtained: 8.2801
Current minimum: 5.7689
Iteration No: 3 started. Evaluating function at random point.
lr: 0.030455926047902763
max_depth: 2
n_estimators: 62
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 10.9611
Function value obtained: 17.4125
Current minimum: 5.7689
Iteration No: 4 started. Evaluating function at random point.
lr: 0.04828884661481365
max_depth: 8
n_estimators: 101
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 49.0062
Function value obtained: 8.3432
Current minimum: 5.7689
Iter

In [22]:
print('Best Value:',res2.fun)
print('Parâmetros:',res2.x)

Best Value: 5.143376913482863
Parâmetros: [0.09287105723056423, 10, 190]


### Aumentando o número de árvores

In [31]:
lr = 0.09497877675572056
max_depth = 10
n_estimators = 200
  
model = LGBMRegressor(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         n_estimators=n_estimators, random_state=40, 
                         n_jobs=6)

vetor_MAPE2,vetor_MSLR2,vetor_RMSE2  = validation1(X,Y,'sales','normal',Date,model)

Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2014-12-31
Data do inicio do Validação  : 2015-01-01 Data do final do Validação   : 2015-02-12
MAPE:  5.13677
RMSLE: 0.00465
RMSE:  472.49775
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-02-11
Data do inicio do Validação  : 2015-02-12 Data do final do Validação   : 2015-03-26
MAPE:  4.89987
RMSLE: 0.00387
RMSE:  408.78426
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-03-25
Data do inicio do Validação  : 2015-03-26 Data do final do Validação   : 2015-05-07
MAPE:  5.31197
RMSLE: 0.00490
RMSE:  601.07882
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-05-06
Data do inicio do Validação  : 2015-05-07 Data do final do Validação   : 2015-06-18
MAPE:  4.74925
RMSLE: 0.00379
RMSE:  459.41336
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-06-17
Data do inicio do Validação  : 2015-06-18 Data do final 

In [32]:
lr = 0.09497877675572056
max_depth = 10
n_estimators = 300
  
model = LGBMRegressor(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         n_estimators=n_estimators, random_state=40, 
                         n_jobs=6)


vetor_MAPE2,vetor_MSLR2,vetor_RMSE2  = validation1(X,Y,'sales','normal',Date,model)

Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2014-12-31
Data do inicio do Validação  : 2015-01-01 Data do final do Validação   : 2015-02-12
MAPE:  5.00707
RMSLE: 0.00444
RMSE:  460.61198
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-02-11
Data do inicio do Validação  : 2015-02-12 Data do final do Validação   : 2015-03-26
MAPE:  4.80146
RMSLE: 0.00372
RMSE:  398.61651
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-03-25
Data do inicio do Validação  : 2015-03-26 Data do final do Validação   : 2015-05-07
MAPE:  5.19266
RMSLE: 0.00472
RMSE:  589.19082
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-05-06
Data do inicio do Validação  : 2015-05-07 Data do final do Validação   : 2015-06-18
MAPE:  4.60992
RMSLE: 0.0036
RMSE:  444.39936
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-06-17
Data do inicio do Validação  : 2015-06-18 Data do final d

In [33]:
lr = 0.09497877675572056
max_depth = 10
n_estimators = 500
  
model = LGBMRegressor(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         n_estimators=n_estimators, random_state=40, 
                         n_jobs=6)

vetor_MAPE2,vetor_MSLR2,vetor_RMSE2  = validation1(X,Y,'sales','normal',Date,model)

Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2014-12-31
Data do inicio do Validação  : 2015-01-01 Data do final do Validação   : 2015-02-12
MAPE:  4.93851
RMSLE: 0.00433
RMSE:  455.10026
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-02-11
Data do inicio do Validação  : 2015-02-12 Data do final do Validação   : 2015-03-26
MAPE:  4.72546
RMSLE: 0.00361
RMSE:  391.1488
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-03-25
Data do inicio do Validação  : 2015-03-26 Data do final do Validação   : 2015-05-07
MAPE:  5.10793
RMSLE: 0.00459
RMSE:  579.69636
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-05-06
Data do inicio do Validação  : 2015-05-07 Data do final do Validação   : 2015-06-18
MAPE:  4.55462
RMSLE: 0.00352
RMSE:  436.52538
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-06-17
Data do inicio do Validação  : 2015-06-18 Data do final d

In [34]:
lr = 0.09497877675572056
max_depth = 10
n_estimators = 700
  
model = LGBMRegressor(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         n_estimators=n_estimators, random_state=40, 
                         n_jobs=6)

vetor_MAPE2,vetor_MSLR2,vetor_RMSE2  = validation1(X,Y,'sales','normal',Date,model)

Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2014-12-31
Data do inicio do Validação  : 2015-01-01 Data do final do Validação   : 2015-02-12
MAPE:  4.93234
RMSLE: 0.00433
RMSE:  455.16337
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-02-11
Data do inicio do Validação  : 2015-02-12 Data do final do Validação   : 2015-03-26
MAPE:  4.6956
RMSLE: 0.00356
RMSE:  388.31351
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-03-25
Data do inicio do Validação  : 2015-03-26 Data do final do Validação   : 2015-05-07
MAPE:  5.07734
RMSLE: 0.00454
RMSE:  576.14794
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-05-06
Data do inicio do Validação  : 2015-05-07 Data do final do Validação   : 2015-06-18
MAPE:  4.52387
RMSLE: 0.00347
RMSE:  433.50908
Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2015-06-17
Data do inicio do Validação  : 2015-06-18 Data do final d

In [ ]:
lr = 0.09497877675572056
max_depth = 10
n_estimators = 1000
  
model = LGBMRegressor(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         n_estimators=n_estimators, random_state=40, 
                         n_jobs=6)


vetor_MAPE2,vetor_MSLR2,vetor_RMSE2  = validation1(X,Y,'sales','normal',Date,model)

Data do inicio do treinamento: 2013-01-03 Data do final do treinamento : 2014-12-31
Data do inicio do Validação  : 2015-01-01 Data do final do Validação   : 2015-02-12


# Avaliando os melhores resultados


### Avaliando os valores gerais
* Legenda:
  * v0 -> Random Forest.
  * v1 -> Lgbm
  * v2 -> Xgbm
---
---
* Mape Geral v0:  5.79653 Std: 0.93450 Erro: 0.41792
* Mape Geral v1:  4.86666 Std: 0.22444 Erro: 0.10037
* Mape Geral v2:  4.82975 Std: 0.27000 Erro: 0.12075
---
---
* Rmsle Geral v0:  0.00574 Std: 0.00181 Erro: 0.00081
* Rmsle Geral v1:  0.00411 Std: 0.00049 Erro: 0.00022
* Rmsle Geral v2:  0.00395 Std: 0.00050 Erro: 0.00022
---
---
* Rmse Geral v0: 597.23134 Std: 157.7021 Erro: 70.52655
* Rmse Geral v1: 482.44171 Std: 73.00971 Erro: 32.65094
* Rmse Geral v2: 476.00876 Std: 73.06339 Erro: 32.67494
---
---



### Conclusão.
 
* Nessa etapa foi treinado e validado o modelo Random Forest, XGBM e LGBM. Entre as suas performances com os dados de validação os valores de MAPE obtidos foram os seguintes:
 * Random forest:   5.79653 % Std: 0.93450 % Error: 0.41792 %
 * LGBM:            4.86666 % Std: 0.22444 % Error: 0.10037 %
 * XGBM:            4.82975 % Std: 0.27000 % Error: 0.12075 %
 
* Nas outras métricas o modelo XGBM também apresentou o melhor resultado. Mas ao observar a medida de erro de todas as medidas é visto que o desempenho do XGBM e LGBM são bem próximos. No caso essa diferença pode estar acontecendo por causa da flutuação estatística.
